In [1]:
import numpy as np
import pandas as pd

In [2]:
def check(st, answ_s):
    for i in answ_s.index:
        if answ_s.loc[i, 'apple'] != st.loc[i, 'apple']:
            print('Error: ', i, ' ', answ_s.loc[i, 'apple'], ' ', st.loc[i, 'apple'])
        if answ_s.loc[i, 'pen'] != st.loc[i, 'pen']:
            print('Error: ', i, ' ', answ_s.loc[i, 'pen'], ' ', st.loc[i, 'pen'])

In [3]:
def day_sell(sell):       
    df = pd.DataFrame(columns=['apple or pen'])
    df['apple or pen'] = sell['sku_num'].apply(lambda x: 'apple' if x.find('ap') == 6 else 'pen') # Ставит вместо транзакции apple/pen
    newdf = pd.crosstab(df.index, df['apple or pen'])    # Новая таблица со столбцами apple, pen, в строках количество за каждый день    
    ds = pd.DataFrame(columns=['apple', 'pen'], index=newdf.index) 
    ds['apple'] = -newdf['apple'].values  # Приводим к нормальному виду вроде
    ds['pen'] = -newdf['pen'].values
    return ds

In [4]:
# №1
def day_store(d_sell, supply):  
    df = pd.concat([supply, d_sell]).sort_index()   # Объединяет 2 таблицы 
    df = df.resample('D').sum()       # Группирует по дням и суммирует (если в день была и поставка и продажа)
    df['apple'] = df['apple'].rolling(str(df.index.size) + 'D').sum()   # ???Чтобы сложить по месяцам
    df['pen'] = df['pen'].rolling(str(df.index.size) + 'D').sum()
    return df

In [5]:
# №2
def stolen(d_sell, supply, inventory):
    m_sell = d_sell.resample('M').sum()    # Группирует по месяцам и суммирует 
    m_supply = supply.resample('M').sum()
    
    stole = pd.DataFrame(columns=['apple', 'pen'], index=m_sell.index) # Сколько товара должно оставаться на складе
    stole['apple'] = m_supply['apple'] + m_sell['apple'] 
    stole['pen'] = m_supply['pen'] + m_sell['pen'] 
    
    res = pd.DataFrame(columns=['apple', 'pen'], index=inventory.index) # Сколько на самом деле остаётся
    res['apple'] = inventory['apple'] - inventory['apple'].shift(1) # Разница между месяцами
    res['pen'] = inventory['pen'] - inventory['pen'].shift(1)       # shift(1) - смещение на 1
    res.loc[res.index[0], ['apple', 'pen']] = inventory.loc[inventory.index[0], 'apple'], inventory.loc[inventory.index[0], 'pen']
    # loc - чтобы обращаться по строке и столбцу
    stole['apple'] -= res['apple']
    stole['pen'] -= res['pen']
    return stole


In [6]:
def filename(state, n):
    sn = chr(ord('0') + n)
    inv = pd.read_csv('./out/input/MS-' + state + sn + '-inventory.csv', index_col='date', parse_dates=True)
    sup = pd.read_csv('./out/input/MS-' + state + sn + '-supply.csv', index_col='date', parse_dates=True)
    sell = pd.read_csv('./out/input/MS-' + state + sn + '-sell.csv', index_col='date', parse_dates=True)
    return inv, sup, sell
    

In [7]:
def ansname(state, n):
    sn = chr(ord('0') + n)
    ans = pd.read_csv('./out/answer/MS-' + state + sn + '-steal.csv', index_col='date', parse_dates=True)
    return ans

In [8]:
state = ['b', 'm', 's']
store = [2, 2, 5]

In [9]:
data_inv = []
data_sup = []
data_sell = []
for i in range(3):
    data_inv.append([])
    data_sup.append([])
    data_sell.append([])
    for j in range(1, store[i] + 1):
        a, b, c = filename(state[i], j)
        data_inv[i].append(a)
        data_sup[i].append(b)
        data_sell[i].append(c)


In [10]:
d_s = []
st = []
steal = []
for i in range(3):
    d_s.append([])
    st.append([])
    steal.append([])
    for j in range(0, store[i]):
        ds = day_sell(data_sell[i][j])
        s = day_store(ds, data_sup[i][j])
        ste = stolen(ds, data_sup[i][j], data_inv[i][j])
        d_s[i].append(ds)
        st[i].append(s)
        steal[i].append(ste)


In [11]:
steal[0][0].head()

,apple,pen
row_0,,
2006-01-31,10.0,11.0
2006-02-28,6.0,6.0
2006-03-31,7.0,6.0
2006-04-30,6.0,14.0
2006-05-31,8.0,1.0


In [17]:
agr_stolen = {}
agr_sold = {}
agr_all = {}

for state_index in range(3):
    #Создаем датафреймы для агрегирования товаров по годам
    agr_stolen[state_index] = pd.DataFrame(columns = ['apple', 'pen'])
    agr_sold[state_index] = pd.DataFrame(columns = ['apple', 'pen'])
    
    for store_index in range(store[state_index]):
        #Собираем данные о товарах по штату, группируем по году
        agr_stolen[state_index] = pd.concat([agr_stolen[state_index], steal[state_index][store_index]]).resample('Y').sum()
        #к пустой таблице присоединяем данные по каждому штату[магазину]
        agr_sold[state_index] = pd.concat([agr_sold[state_index], d_s[state_index][store_index]]).resample('Y').sum()
    #Переименовываем столбцы
    agr_stolen[state_index] = agr_stolen[state_index].rename(columns = {'apple': 'apple_stolen', 'pen': 'pen_stolen'})
    agr_sold[state_index] = agr_sold[state_index].rename(columns = {'apple': 'apple_sold', 'pen': 'pen_sold'})
    #Соединяем таблицы с данными о проданных и украденных товарах
    agr_all[state_index] = pd.merge(agr_sold[state_index], agr_stolen[state_index], left_index=True, right_index=True)
    
    agr_all[state_index]['apple_sold'] = abs(agr_all[state_index]['apple_sold']) 
    agr_all[state_index]['pen_sold'] = abs(agr_all[state_index]['pen_sold'])
    agr_all[state_index]['apple_stolen'] = agr_all[state_index]['apple_stolen'].apply(lambda x: int(x))
    agr_all[state_index]['pen_stolen'] = agr_all[state_index]['pen_stolen'].apply(lambda x: int(x))
    #Добавляем информацию о штате
    agr_all[state_index]['state'] = pd.Series([state[state_index]] * len(agr_all[state_index].index), index=agr_all[state_index].index)
    cols = ['state', 'apple_sold', 'apple_stolen', 'pen_sold', 'pen_stolen'] 
    agr_all[state_index] = agr_all[state_index][cols] #Меняем порядок столбцов
#Соединяем данные по штатам в одну таблицу
agregated = pd.concat([agr_all[0], agr_all[1], agr_all[2]]).sort_index()
agregated.index.name = 'date'
agregated.index = agregated.index.year

print(agregated)


     state  apple_sold  apple_stolen  pen_sold  pen_stolen
date                                                      
2006     b     1362682           156    105110         182
2006     s      169412           132     18167         121
2006     m      619912           130     32356         158
2007     b     1362562           136    103878         152
2007     s      168868           117     18474          90
2007     m      618954           124     32378         104
2008     b     1370220           156    104064         146
2008     s      171013            93     18251         120
2008     m      622326           134     32282         116
2009     b     1364592           162    104822         192
2009     s      168752           143     18211         130
2009     m      619158           128     32376         132
2010     s      169077           156     18451         133
2010     m      619888           126     31872         116
2010     b     1360822           136    105200         1

In [13]:
for i in range(3):
    for j in range(1, store[i] + 1):
        answ = ansname(state[i], j)
        check(steal[i][j-1], answ)